# Art and Architecture Thesaurus Concepts (AATC) Controlled Vocabulary

Code (mostly SPARQL queries) to generate the AATC controlled vocabulary. (For more documentation see README.md)

## On Reusing Named Entities in new Concept Schema

https://www.w3.org/TR/skos-primer/#secextension  *3.2 Re-using and Extending Concept Schemes* mentions that
> Linking concepts by means of mappings is not the only way to interlink concept schemes. The use of URIs on the Semantic Web allows resources to be shared and reused in a distributed fashion. As a result it is possible for a SKOS concept to participate in several concept schemes at the same time. For example, a SKOS publisher can choose to locally extend an existing concept scheme by declaring any new concepts that may be needed and simply linking to concepts that have already been defined in the existing scheme.
> 
> A new concept scheme can re-use existing concepts using the skos:inScheme property. 

Reusing existing URIs will leverage the existing meaning and relationships established within the original term definition.
 However, attention should be payed to maintain URI and its meaning, ensuring that the meaning of the concept URI remains consistent with its origin definition


## New Schema: aatc

aatc URI https://vocabularies.dans.knaw.nl/aatconcepts/ is created based on URL and convention of DANS Skomos server

In [1]:
# imports SPARQL prefixes and functions defs

from pprint import pprint
from SPARQLWrapper import SPARQLWrapper, JSON, TURTLE, CSV 

prefixes = '''    
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX aat: <http://vocab.getty.edu/aat/>
PREFIX gvp: <http://vocab.getty.edu/ontology#> 
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX skosxl: <http://www.w3.org/2008/05/skos-xl#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX aatc: <http://vocabularies.dans.knaw.nl/aatconcepts/>
'''    


def sparql_query(query, format):
    formats = {"json": JSON, "turtle": TURTLE, "csv": CSV}
    f_ = formats[format]
    endpoint = "http://vocab.getty.edu/sparql"
    sparql = SPARQLWrapper(endpoint)

    query = prefixes + query     
    sparql.setQuery(query)

    sparql.setReturnFormat(f_)
    results = sparql.query().convert()    
    # sparql.setReturnFormat(XML)
    # results = sparql.query()
    return results
    # # Print the results
    # print("Subject ID: ", subjectID)
    # 


def print_sparql_results(results):
    for row in results["results"]["bindings"]:
        return (row)


In [ ]:
# SELECT QUERY: GETTY AAT Concepts with labels in EN and NL 
# lang: @en-US is converted to @en 

pref_label_query = '''
SELECT ?concept ?label_literal_nl ?label_literal_en
WHERE {
  ?concept a gvp:Concept ;
          skos:inScheme aat: ;
          skosxl:prefLabel ?preflabel . 
            
  { ?preflabel  dcterms:language aat:300388277 ;
                skosxl:literalForm ?label_literal_en. } # lang: @en
  
  UNION
  
  { ?preflabel  dcterms:language aat:300387822 ;
                skosxl:literalForm ?label_literal_en_us . 
    BIND (STRLANG(STR(?label_literal_en_us), 'en') AS  ?label_literal_en)} # lang: @en-US - converts to @en
  
  UNION  
  
  { ?preflabel  dcterms:language aat:300388256 ;
                skosxl:literalForm ?label_literal_nl . } # lang: @nl 

}
LIMIT 20
''' 

subjs_w_preflable = sparql_query(query=pref_label_query, format='json')
pprint(subjs_w_preflable)

In [ ]:
# CONSTRUCT QUERY: GETTY AAT Concepts - slim  
# AAT gvp:concepts with labels in EN and NL become a skos:Concept with rdfs:label @en @nl
# lang: @en-US is converted to @en 
# AAT concept become members (skos:inScheme) of AATC skos:ConceptScheme
# Output: aatc.ttl
from datetime import date

today = date.today().strftime("%Y-%m-%d")

pref_label_query = '''
CONSTRUCT {
    <http://vocabularies.dans.knaw.nl/aatconcepts> a skos:ConceptScheme ;
        dct:title "The Art and Architecture Thesaurus Concepts"@en ;
        rdfs:label "The Art and Architecture Thesaurus Concepts"@en ;
        rdfs:comment "The Art and Architecture Thesaurus Concepts (AATC) is a SKOS concept scheme that restructures the concepts from the Art and Architecture Thesaurus (AAT) into a flat controlled vocabulary, and excludes non-concepts, such as facets, hierarchies and guide terms. The result is a controlled vocabulary where each term is a skos:Concept, member of aatc: skos:ConceptScheme, with English(@en) and Dutch(@nl) labels. Original URI are kept, and users are encourage to find more information and reference the term through its URI. AATC development was motivated by the need to index AAT terms in a Skosmos server, so that AAT terms could be used easily queried via the Skosmos API and easily used by software applications to classify data with AAT terms"@en ;
        dct:creator <https://ror.org/008pnp284> , <https://orcid.org/0000-0002-7839-3698> ;
        dct:created "%s"^^xsd:date ;
        dcterms:license <http://opendatacommons.org/licenses/by/1.0/> .
        
    ?concept a skos:Concept ;
        skos:inScheme <http://vocabularies.dans.knaw.nl/aatconcepts> ;
        skos:prefLabel ?label_literal_en ;
        skos:prefLabel ?label_literal_nl ;
        dcterms:issued ?issuedDate .
}

WHERE {
    ?concept a gvp:Concept ;
        skos:inScheme aat: ;
        skosxl:prefLabel ?preflabel .
        
    OPTIONAL {?concept dcterms:issued ?issuedDate .}

    { ?preflabel  dcterms:language aat:300388277 ; skosxl:literalForm ?label_literal_en.  } # lang: @en
                 
    UNION

    { ?preflabel  dcterms:language aat:300387822 ; skosxl:literalForm ?label_literal_en_us . 
      BIND (STRLANG(STR(?label_literal_en_us), 'en') AS  ?label_literal_en)} # lang: @en-US - converts to @en
    
    UNION
    
    { ?preflabel  dcterms:language aat:300388256 ; skosxl:literalForm ?label_literal_nl . } # lang: @nl 
    FILTER( STRSTARTS(str(?concept), str(aat:)) )
}
ORDER BY ?concept
LIMIT 40
''' % today

construct_results = sparql_query(query=pref_label_query, format='turtle')
print(construct_results)
with open('aatc-nb.ttl', 'wb') as att_subjects_f:
    att_subjects_f.write(construct_results)


In [ ]:
from pyshacl import validate
from rdflib import Graph

sg = Graph()
sg.parse("aatc_shacl.ttl")


r = validate(construct_results,
      shacl_graph=sg,
      ont_graph=None,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)
conforms, results_graph, results_text = r